<a target="_blank" href="https://colab.research.google.com/github/sergiopaniego/RAG_local_tutorial/blob/main/youtube_rag.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Example of RAG with a Youtube video

In this example, we download the trascription of a Youtube video and use an LLM for extracting information from that video.

**Please, complete the example_rag.ipynb first to get more insight.**

Let's go!

<p align="center">
  <img src="https://cdn.analyticsvidhya.com/wp-content/uploads/2023/07/langchain3.png" alt="Langchain Logo" width="20%">
  <img src="https://bookface-images.s3.amazonaws.com/logos/ee60f430e8cb6ae769306860a9c03b2672e0eaf2.png" alt="Ollama Logo" width="20%">
  <img src="https://upload.wikimedia.org/wikipedia/commons/e/ef/Youtube_logo.png" alt="Youtube Logo" width="20%">
</p>

Sources:

* https://github.com/svpino/llm

# Requirements

* Ollama installed locally

### Install the dependencies

In [ ]:
!pip3 install langchain
!pip3 install langchain_pinecone
!pip3 install langchain[docarray]
!pip3 install docarray
!pip3 install pypdf

In [ ]:
!pip3 install youtube_transcript_api

# Let's download an example transcript from a YT video

You can change the id of the video to download other video transcriptions.
We save the contect to a file

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

srt = YouTubeTranscriptApi.get_transcript("pxiP-HJLCx0") # CHANGE THE ID OF THE VIDEO

with open("./files/youtube_transcription.txt", "a") as file:
    for i in srt:
        file.write(i['text'])

# Select the LLM model to use

The model must be downloaded locally to be used, so if you want to run llama3, you should run:

```

ollama pull llama3

```

Check the list of models available for Ollama here: https://ollama.com/library

# We instantiate the model and the embeddings

In [ ]:
#MODEL = "gpt-3.5-turbo"
#MODEL = "mixtral:8x7b"
#MODEL = "gemma:7b"
#MODEL = "llama2"
MODEL = "llama3" # https://ollama.com/library/llama3

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

# We load the transcription previously saved using TextLoader

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./files/youtube_transcription.txt")
text_documents = loader.load()
text_documents

# We explit the document into chunks

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
text_splitter.split_documents(text_documents)[:5]

In [ ]:
text_documents

# Store the PDF in a vector space.

From Langchain docs:

`DocArrayInMemorySearch is a document index provided by Docarray that stores documents in memory. It is a great starting point for small datasets, where you may not want to launch a database server.`

The execution time of the following block depends on the complexity and longitude of the PDF provided. Try to keep it small and simple for the example.

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(text_documents, embedding=embeddings)

In [ ]:
retriever = vectorstore.as_retriever()

# We instantiate the parser

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

# Generate the conversation template

In [ ]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, answer with "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

# We can now extract the information from the video!

In [ ]:
retrieved_context = retriever.invoke("laptop")
questions = [
    "Which is the best laptop for students?",
    "How much is a laptop worth?",
    "Make a summary of the video"
]

for question in questions:
    formatted_prompt = prompt.format(context=retrieved_context, question=question)
    response_from_model = model.invoke(formatted_prompt)
    parsed_response = parser.parse(response_from_model)

    print(f"Question: {question}")
    print(f"Answer: {parsed_response}")
    print()